In [ ]:
## Data Initializing

In [1]:
import pandas as pd
import numpy as np
import data_handler as dh
import model as md
import time
import matplotlib.pyplot as plt
from matplotlib import rc
import pickle

In [ ]:
import importlib
importlib.reload(dh)
importlib.reload(md)

In [15]:
import numba
numba.__version__

'0.49.0'

# Stable Dynamics Model

parameter $\mu = 0$

In [2]:
sd_save = 'stable_dynamics_results/'
net_name = 'Anaheim_net.tntp'
trips_name = 'Anaheim_trips.tntp'

In [3]:
handler = dh.DataHandler()
graph_correspondences, total_od_flow = handler.GetGraphCorrespondences(trips_name)
graph_data = handler.GetGraphData(net_name, columns = ['init_node', 'term_node', 'capacity', 'free_flow_time'])
init_capacities = graph_data['graph_table']['capacity'] * 2.5

## Base flows
First of all, we should find admissible set of flows on the transport graph. It is required for defining duality gap.

In [4]:
alpha = 0.8
graph_data['graph_table']['capacity'] = init_capacities * alpha
model = md.Model(graph_data, graph_correspondences,
                 total_od_flow, mu = 0)

graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,False,117,True,18000.0,1.090458
1,2,False,87,True,18000.0,1.090458
2,3,False,74,True,18000.0,1.090458
3,4,False,233,True,18000.0,1.090458
4,5,False,165,True,18000.0,1.090458


In [6]:
assert(model.mu == 0)
max_iter = 1000

solver_kwargs = {'eps': 1e-4,
                 'max_iter': max_iter, 'stop_crit': 'max_iter',
                 'verbose': True, 'verbose_step': 200, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'ustm', composite = True,
                                solver_kwargs = solver_kwargs,
                                base_flows = alpha * graph_data['graph_table']['capacity'])
                                #base_flows here doesnt define anything now
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic
#with open(sd_save + 'anaheim_result_' + 'ustm' + '_base_flows_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
#    pickle.dump(result, f)

Composite optimization...
Oracles created...
Universal similar triangles method...
Primal_init = 1.24813e+06
Dual_init = -1.24813e+06
Duality_gap_init = 3.717e+06

Iterations number: 200
Inner iterations number: 395
Primal_func_value = 1.24922e+06
Dual_func_value = -1.24922e+06
Duality_gap = 30927.5
Duality_gap / Duality_gap_init = 0.00832054

Iterations number: 400
Inner iterations number: 796
Primal_func_value = 1.24922e+06
Dual_func_value = -1.24922e+06
Duality_gap = 15524.7
Duality_gap / Duality_gap_init = 0.00417667

Iterations number: 600
Inner iterations number: 1196
Primal_func_value = 1.24922e+06
Dual_func_value = -1.24922e+06
Duality_gap = 11378.4
Duality_gap / Duality_gap_init = 0.00306118

Iterations number: 800
Inner iterations number: 1597
Primal_func_value = 1.24922e+06
Dual_func_value = -1.24922e+06
Duality_gap = 8217.62
Duality_gap / Duality_gap_init = 0.00221082

Iterations number: 1000
Inner iterations number: 1998
Primal_func_value = 1.24922e+06
Dual_func_value = -1

In [47]:
np.max(result['flows'] / init_capacities)

0.7556777777777777

## Universal Similar Triangles

In [14]:
graph_data['graph_table']['capacity'] = init_capacities
graph_data['graph_table'].head()

,init_node,init_node_thru,term_node,term_node_thru,capacity,free_flow_time
0,1,False,117,True,22500.0,1.090458
1,2,False,87,True,22500.0,1.090458
2,3,False,74,True,22500.0,1.090458
3,4,False,233,True,22500.0,1.090458
4,5,False,165,True,22500.0,1.090458


In [ ]:
assert(model.mu == 0)
max_iter = 10000
for index, eps_abs in enumerate(epsilons):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter,
                     'verbose' : True, 'verbose_step': 2000, 'save_history' : True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustm', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(sd_save + 'anaheim_result_' + 'ustm' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
        pickle.dump(result, f)

In [ ]:
assert(model.mu == 0)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    if index < len(epsilons) - 1:
        continue
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter,
                     'verbose' : True, 'verbose_step': 4000, 'save_history' : True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ustm', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(sd_save + 'anaheim_result_' + 'ustm' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
        pickle.dump(result, f)

## Universal Gradient Descent

In [ ]:
#weighted t added
assert(model.mu == 0)
max_iter = 20000
for index, eps_abs in enumerate(epsilons):
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter,
                     'verbose' : True, 'verbose_step': 2000, 'save_history' : True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ugd', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')
    
    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(sd_save + 'anaheim_result_' + 'ugd' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
        pickle.dump(result, f)

In [ ]:
#weighted t added
assert(model.mu == 0)
max_iter = 40000
for index, eps_abs in enumerate(epsilons):
    if index < 3:
        continue
    print('eps_abs =', eps_abs)
    solver_kwargs = {'eps_abs': eps_abs,
                     'max_iter': max_iter,
                     'verbose' : True, 'verbose_step': 4000, 'save_history' : True}
    tic = time.time()
    result = model.find_equilibrium(solver_name = 'ugd', composite = True, solver_kwargs = solver_kwargs)
    toc = time.time()
    print('Elapsed time: {:.0f} sec'.format(toc - tic))
    print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
    print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')
    
    result['eps_abs'] = eps_abs
    result['elapsed_time'] = toc - tic
    with open(sd_save + 'anaheim_result_' + 'ugd' + '_eps_abs_' + str(index) + '_SD.pickle', 'wb') as f:
        pickle.dump(result, f)

## Weighted Dual Averages

### Composite optimization

In [ ]:
assert(model.mu == 0)
max_iter = 40000
solver_kwargs = {'eps_abs': - np.inf,
                 'max_iter': max_iter,
                 'verbose': True, 'verbose_step': 4000, 'save_history': True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'wda', composite = True, solver_kwargs = solver_kwargs)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic
with open(sd_save + 'anaheim_result_' + 'wda' + '_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
    pickle.dump(result, f)

### Non-composite optimization

In [ ]:
assert(model.mu == 0)
max_iter = 40000
solver_kwargs = {'eps_abs': - np.inf,
                 'max_iter': max_iter,
                 'verbose': True, 'verbose_step': 4000, 'save_history' : True}
tic = time.time()
result = model.find_equilibrium(solver_name = 'wda', composite = False, solver_kwargs = solver_kwargs)
toc = time.time()
print('Elapsed time: {:.0f} sec'.format(toc - tic))
print('Time ratio =', np.max(result['times'] / graph_data['graph_table']['free_flow_time']))
print('Flow excess =', np.max(result['flows'] / graph_data['graph_table']['capacity']) - 1, end = '\n\n')

result['elapsed_time'] = toc - tic
with open(sd_save + 'anaheim_result_' + 'wda_noncomposite' + '_max_iter_' + str(max_iter) + '_SD.pickle', 'wb') as f:
    pickle.dump(result, f)